In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from collections import defaultdict


In [2]:
myData = pd.read_csv("train-data.csv")
myData.drop(columns=['New_Price'], inplace=True)
myData = myData.dropna()
def convert_mileage(mileage):
    if 'km/kg' in mileage:
        # Convert km/kg to kmpl
        kmpl = float(mileage.split()[0]) * 0.453592 / 1.609344
        return round(kmpl, 2)
    else:
        # Remove 'kmpl' and convert to float
        return float(mileage.split()[0])

# Define a conversion function to convert the power to bhp
def convert_power(power):
    # Remove 'bhp' and convert to float
    return float(power.split()[0])


def process_df(myData):
    myData['Cars'] = myData['Name'].str.split(" ").str[0] 
    myData['Submodel']=myData['Name'].str.split(" ").str[1]
    myData.drop(columns=['Name'], inplace=True)

        # Apply the conversion functions to the 'Mileage' and 'Power' columns

    return myData


myData = process_df(myData)
myData['Mileage'] = myData['Mileage'].apply(convert_mileage)
myData['Power'] = myData['Power'].apply(convert_power)
myData['Engine'] = myData['Engine'].apply(lambda x: int(x.split(" ")[0]))
myData.head()


,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,Cars,Submodel
0,Mumbai,2010,72000,CNG,Manual,First,7.50,998,58.16,5.0,1.75,Maruti,Wagon
1,Pune,2015,41000,Diesel,Manual,First,19.67,1582,126.20,5.0,12.50,Hyundai,Creta
2,Chennai,2011,46000,Petrol,Manual,First,18.20,1199,88.70,5.0,4.50,Honda,Jazz
3,Chennai,2012,87000,Diesel,Manual,First,20.77,1248,88.76,7.0,6.00,Maruti,Ertiga
4,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968,140.80,5.0,17.74,Audi,A4


In [3]:

columns=['Location', 'Fuel_Type', 'Transmission', 'Owner_Type', 'Cars', 'Submodel', 'Year']

train_encoders = defaultdict(None)

def train_one_hot_encoder(feature_name, train_data):
    enc = OneHotEncoder(handle_unknown='ignore')
    enc.fit(train_data.reshape(-1, 1))
    train_encoders[feature_name] = enc


def convert_df(df):
    df = df.dropna()

    for i in columns:
        enc= train_encoders[i]
        print(i,type(i),enc.categories_[0])
        df[enc.categories_[0]] = enc.transform(df[i].values.reshape(-1, 1)).toarray()
        
        df.drop(columns=[i], inplace=True)
    return df
# Train one-hot encoders
for i in columns:
    train_one_hot_encoder(i, myData[i].values)


myData = convert_df(myData)

Location <class 'str'> ['Ahmedabad' 'Bangalore' 'Chennai' 'Coimbatore' 'Delhi' 'Hyderabad'
 'Jaipur' 'Kochi' 'Kolkata' 'Mumbai' 'Pune']
Fuel_Type <class 'str'> ['CNG' 'Diesel' 'LPG' 'Petrol']
Transmission <class 'str'> ['Automatic' 'Manual']
Owner_Type <class 'str'> ['First' 'Fourth & Above' 'Second' 'Third']
Cars <class 'str'> ['Ambassador' 'Audi' 'BMW' 'Bentley' 'Chevrolet' 'Datsun' 'Fiat' 'Force'
 'Ford' 'Honda' 'Hyundai' 'ISUZU' 'Isuzu' 'Jaguar' 'Jeep' 'Lamborghini'
 'Land' 'Mahindra' 'Maruti' 'Mercedes-Benz' 'Mini' 'Mitsubishi' 'Nissan'
 'Porsche' 'Renault' 'Skoda' 'Smart' 'Tata' 'Toyota' 'Volkswagen' 'Volvo']
Submodel <class 'str'> ['1' '1000' '3' '5' '6' '7' '800' 'A' 'A-Star' 'A3' 'A4' 'A6' 'A7' 'A8'
 'Accent' 'Accord' 'Alto' 'Amaze' 'Ameo' 'Aspire' 'Aveo' 'Avventura' 'B'
 'BR-V' 'BRV' 'Baleno' 'Beat' 'Beetle' 'Bolero' 'Bolt' 'Boxster' 'Brio'
 'C-Class' 'CLA' 'CLS-Class' 'CR-V' 'Camry' 'Captiva' 'Captur' 'Cayenne'
 'Cayman' 'Cedia' 'Celerio' 'Ciaz' 'City' 'Civic' 'Classic' 'Clu

C:\Users\bobba\AppData\Local\Temp\ipykernel_16704\2309633390.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[enc.categories_[0]] = enc.transform(df[i].values.reshape(-1, 1)).toarray()
C:\Users\bobba\AppData\Local\Temp\ipykernel_16704\2309633390.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[enc.categories_[0]] = enc.transform(df[i].values.reshape(-1, 1)).toarray()
C:\Users\bobba\AppData\Local\Temp\ipykernel_16704\2309633390.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of

In [4]:
myData

,Kilometers_Driven,Mileage,Engine,Power,Seats,Price,Ahmedabad,Bangalore,Chennai,Coimbatore,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,72000,7.50,998,58.16,5.0,1.75,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,41000,19.67,1582,126.20,5.0,12.50,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,46000,18.20,1199,88.70,5.0,4.50,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,87000,20.77,1248,88.76,7.0,6.00,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,40670,15.20,1968,140.80,5.0,17.74,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6014,27365,28.40,1248,74.00,5.0,4.75,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6015,100000,24.40,1120,71.00,5.0,4.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6016,55000,14.00,2498,112.00,8.0,2.90,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6017,46000,18.90,998,67.10,5.0,2.65,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:

from sklearn.model_selection import train_test_split
x = myData.drop(['Price'], axis =1)
y = myData['Price']

import pandas as pd
import keras
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential

import numpy as np
from copy import deepcopy

X_data = deepcopy(np.array(x))
y_data = deepcopy(np.array(y).reshape(-1, 1))

print("Input data: \n",X_data[0])
print("\nCar price : \n",y_data[0])

x_scaler = StandardScaler()
y_scaler = StandardScaler()

print("Input data: \n",X_data[0])
print("\nCar price : \n",y_data[0])


X_data_scaled = x_scaler.fit_transform(X_data)
y_data_scaled = y_scaler.fit_transform(y_data)

print("Input data: \n",X_data_scaled[0])
print("\nCar price : \n",y_data_scaled[0])
x_train, x_test, y_train, y_test = train_test_split(X_data_scaled, y_data_scaled, test_size=0.1)

print("x data size : ", len(x))
print("y data size : ", len(y))
print("x_test data size : ", len(x_test))
print("y_test data size : ", len(y_test))
print("x_train data size : ", len(x_train))
print("y_train data size : ", len(y_train))

Input data: 
 [7.200e+04 7.500e+00 9.980e+02 5.816e+01 5.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 1.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 1.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.

In [6]:
model = Sequential()
model.add(Dense(180, activation='relu', input_shape=(None, 286)))
model.add(Dense(140, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='mse', metrics=['mse'])
graph = model.fit(x_train, y_train, epochs=100, batch_size = 100, validation_split = 0.1)

Epoch 1/100
49/49 [==============================] - 1s 9ms/step - loss: 0.4559 - mse: 0.4559 - val_loss: 0.3702 - val_mse: 0.3702
Epoch 2/100
49/49 [==============================] - 0s 4ms/step - loss: 0.2395 - mse: 0.2395 - val_loss: 0.2806 - val_mse: 0.2806
Epoch 3/100
49/49 [==============================] - 0s 4ms/step - loss: 0.2175 - mse: 0.2175 - val_loss: 0.2593 - val_mse: 0.2593
Epoch 4/100
49/49 [==============================] - 0s 4ms/step - loss: 0.1859 - mse: 0.1859 - val_loss: 0.2279 - val_mse: 0.2279
Epoch 5/100
49/49 [==============================] - 0s 4ms/step - loss: 0.1145 - mse: 0.1145 - val_loss: 0.2621 - val_mse: 0.2621
Epoch 6/100
49/49 [==============================] - 0s 4ms/step - loss: 0.0836 - mse: 0.0836 - val_loss: 0.2378 - val_mse: 0.2378
Epoch 7/100
49/49 [==============================] - 0s 4ms/step - loss: 0.0704 - mse: 0.0704 - val_loss: 0.2451 - val_mse: 0.2451
Epoch 8/100
49/49 [==============================] - 0s 4ms/step - loss: 0.0586 - m

49/49 [==============================] - 0s 4ms/step - loss: 0.0082 - mse: 0.0082 - val_loss: 0.3095 - val_mse: 0.3095
Epoch 64/100
49/49 [==============================] - 0s 4ms/step - loss: 0.0093 - mse: 0.0093 - val_loss: 0.3109 - val_mse: 0.3109
Epoch 65/100
49/49 [==============================] - 0s 4ms/step - loss: 0.0090 - mse: 0.0090 - val_loss: 0.2877 - val_mse: 0.2877
Epoch 66/100
49/49 [==============================] - 0s 4ms/step - loss: 0.0090 - mse: 0.0090 - val_loss: 0.3032 - val_mse: 0.3032
Epoch 67/100
49/49 [==============================] - 0s 4ms/step - loss: 0.0078 - mse: 0.0078 - val_loss: 0.2900 - val_mse: 0.2900
Epoch 68/100
49/49 [==============================] - 0s 4ms/step - loss: 0.0078 - mse: 0.0078 - val_loss: 0.3093 - val_mse: 0.3093
Epoch 69/100
49/49 [==============================] - 0s 4ms/step - loss: 0.0072 - mse: 0.0072 - val_loss: 0.3114 - val_mse: 0.3114
Epoch 70/100
49/49 [==============================] - 0s 4ms/step - loss: 0.0070 - mse: 0

In [7]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBClassifier
from xgboost.sklearn import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor,BaggingRegressor, RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn import metrics

In [8]:
Acc = pd.DataFrame(index=None, columns=['model','Root Mean Squared  Error','Accuracy on Traing set','Accuracy on Testing set'])

In [9]:
    
regressors = [['DecisionTreeRegressor',DecisionTreeRegressor()],
              ['XGBRegressor', XGBRegressor()],
              ['RandomForestRegressor', RandomForestRegressor()],
              ['MLPRegressor',MLPRegressor()],
              ['AdaBoostRegressor',AdaBoostRegressor()],
              ['ExtraTreesRegressor',ExtraTreesRegressor()]]


models={}
for mod in regressors:
  name = mod[0]
  model1 = mod[1]
  model1.fit(x_train,y_train)
  y_pred = model1.predict(x_test)
  models[name]=model1
  RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
  ATrS =  model1.score(x_train,y_train)
  ATeS = model1.score(x_test,y_test)

  Acc=Acc.append(pd.Series({'model':name, 'Root Mean Squared  Error': RMSE,'Accuracy on Traing set':ATrS,'Accuracy on Testing set':ATeS}),ignore_index=True )

C:\Users\bobba\AppData\Local\Temp\ipykernel_16704\3477262338.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Acc=Acc.append(pd.Series({'model':name, 'Root Mean Squared  Error': RMSE,'Accuracy on Traing set':ATrS,'Accuracy on Testing set':ATeS}),ignore_index=True )
C:\Users\bobba\AppData\Local\Temp\ipykernel_16704\3477262338.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Acc=Acc.append(pd.Series({'model':name, 'Root Mean Squared  Error': RMSE,'Accuracy on Traing set':ATrS,'Accuracy on Testing set':ATeS}),ignore_index=True )
C:\Users\bobba\AppData\Local\Temp\ipykernel_16704\3477262338.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model1.fit(x_train,y_train)
C:\Users\bobba\AppData\Local\

In [10]:
Acc.sort_values(by='Accuracy on Testing set')

,model,Root Mean Squared Error,Accuracy on Traing set,Accuracy on Testing set
4,AdaBoostRegressor,0.941418,0.163934,0.364572
3,MLPRegressor,0.729553,0.963875,0.618393
0,DecisionTreeRegressor,0.520838,0.999996,0.805505
2,RandomForestRegressor,0.414730,0.982303,0.876680
1,XGBRegressor,0.411093,0.990896,0.878833
5,ExtraTreesRegressor,0.389832,0.999996,0.891042


In [11]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_test, y_pred)

0.15196918317815977

In [12]:
user_sample={'Name':['Hyundai Creta 1.6 CRDi SX Option'], 'Location':['Pune'],
             'Year':[2015], 
             'Kilometers_Driven':[41000], 'Fuel_Type':['Diesel'],
       'Transmission':['Manual'], 'Owner_Type':['First'], 
       'Mileage':[19.67], 'Engine':[1582], 'Power':[126.2], 'Seats':[5]}


In [13]:
def predict_final(user_df):

    user_df = process_df(user_df)
    user_df = convert_df(user_df)
    data = x_scaler.transform(user_df.values.reshape(1,user_df.shape[1]))
    temp={}
    for key,model in models.items():
        
        temp[key]=y_scaler.inverse_transform([model.predict(data)])[0][0]
    return temp

In [14]:
user_df=pd.DataFrame(user_sample)

price = predict_final(user_df)

C:\Users\bobba\AppData\Local\Temp\ipykernel_16704\2309633390.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[enc.categories_[0]] = enc.transform(df[i].values.reshape(-1, 1)).toarray()
C:\Users\bobba\AppData\Local\Temp\ipykernel_16704\2309633390.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[enc.categories_[0]] = enc.transform(df[i].values.reshape(-1, 1)).toarray()
C:\Users\bobba\AppData\Local\Temp\ipykernel_16704\2309633390.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of

Location <class 'str'> ['Ahmedabad' 'Bangalore' 'Chennai' 'Coimbatore' 'Delhi' 'Hyderabad'
 'Jaipur' 'Kochi' 'Kolkata' 'Mumbai' 'Pune']
Fuel_Type <class 'str'> ['CNG' 'Diesel' 'LPG' 'Petrol']
Transmission <class 'str'> ['Automatic' 'Manual']
Owner_Type <class 'str'> ['First' 'Fourth & Above' 'Second' 'Third']
Cars <class 'str'> ['Ambassador' 'Audi' 'BMW' 'Bentley' 'Chevrolet' 'Datsun' 'Fiat' 'Force'
 'Ford' 'Honda' 'Hyundai' 'ISUZU' 'Isuzu' 'Jaguar' 'Jeep' 'Lamborghini'
 'Land' 'Mahindra' 'Maruti' 'Mercedes-Benz' 'Mini' 'Mitsubishi' 'Nissan'
 'Porsche' 'Renault' 'Skoda' 'Smart' 'Tata' 'Toyota' 'Volkswagen' 'Volvo']
Submodel <class 'str'> ['1' '1000' '3' '5' '6' '7' '800' 'A' 'A-Star' 'A3' 'A4' 'A6' 'A7' 'A8'
 'Accent' 'Accord' 'Alto' 'Amaze' 'Ameo' 'Aspire' 'Aveo' 'Avventura' 'B'
 'BR-V' 'BRV' 'Baleno' 'Beat' 'Beetle' 'Bolero' 'Bolt' 'Boxster' 'Brio'
 'C-Class' 'CLA' 'CLS-Class' 'CR-V' 'Camry' 'Captiva' 'Captur' 'Cayenne'
 'Cayman' 'Cedia' 'Celerio' 'Ciaz' 'City' 'Civic' 'Classic' 'Clu

C:\Users\bobba\AppData\Local\Temp\ipykernel_16704\2309633390.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[enc.categories_[0]] = enc.transform(df[i].values.reshape(-1, 1)).toarray()
C:\Users\bobba\AppData\Local\Temp\ipykernel_16704\2309633390.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[enc.categories_[0]] = enc.transform(df[i].values.reshape(-1, 1)).toarray()
C:\Users\bobba\AppData\Local\Temp\ipykernel_16704\2309633390.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of

In [15]:
price

{'DecisionTreeRegressor': 12.5,
 'XGBRegressor': 11.490113612894325,
 'RandomForestRegressor': 12.205000000000002,
 'MLPRegressor': 13.211322095853884,
 'AdaBoostRegressor': 16.376031659388637,
 'ExtraTreesRegressor': 12.500000000000004}

# Backend

In [16]:
from flask import Flask, request, jsonify
import pandas as pd

app = Flask(__name__)
from flask_cors import CORS

cors = CORS(app)

# user_sample={'Name':['Hyundai Creta 1.6 CRDi SX Option'], 'Location':['Pune'],
#              'Year':[2015], 
#              'Kilometers_Driven':[41000], 'Fuel_Type':['Diesel'],
#        'Transmission':['Manual'], 'Owner_Type':['First'], 
#        'Mileage':[19.67], 'Engine':[1582], 'Power':[126.2], 'Seats':[5]}


@app.route('/submit-form', methods=['POST'])
def submit_form():
    data = request.json
    df = pd.DataFrame(columns=['Name','Location','Year','Kilometers_Driven','Fuel_Type','Transmission','Owner_Type','Mileage','Engine','Power','Seats'])
    df = df.append(data, ignore_index=True)
    response_data = predict_final(df)
    return jsonify(response_data)

In [ ]:
#running the server
def main():
    app.run()

if __name__ == '__main__':
    main()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [20/Apr/2023 18:34:53] "OPTIONS /submit-form HTTP/1.1" 200 -
C:\Users\bobba\AppData\Local\Temp\ipykernel_16704\2147629206.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\bobba\AppData\Local\Temp\ipykernel_16704\2309633390.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[enc.categories_[0]] = enc.transform(df[i].values.reshape(-1, 1)).toarray()
C:\Users\bobba\AppData\Local\Temp\ipykernel_16704\2309633390.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor

Location <class 'str'> ['Ahmedabad' 'Bangalore' 'Chennai' 'Coimbatore' 'Delhi' 'Hyderabad'
 'Jaipur' 'Kochi' 'Kolkata' 'Mumbai' 'Pune']
Fuel_Type <class 'str'> ['CNG' 'Diesel' 'LPG' 'Petrol']
Transmission <class 'str'> ['Automatic' 'Manual']
Owner_Type <class 'str'> ['First' 'Fourth & Above' 'Second' 'Third']
Cars <class 'str'> ['Ambassador' 'Audi' 'BMW' 'Bentley' 'Chevrolet' 'Datsun' 'Fiat' 'Force'
 'Ford' 'Honda' 'Hyundai' 'ISUZU' 'Isuzu' 'Jaguar' 'Jeep' 'Lamborghini'
 'Land' 'Mahindra' 'Maruti' 'Mercedes-Benz' 'Mini' 'Mitsubishi' 'Nissan'
 'Porsche' 'Renault' 'Skoda' 'Smart' 'Tata' 'Toyota' 'Volkswagen' 'Volvo']
Submodel <class 'str'> ['1' '1000' '3' '5' '6' '7' '800' 'A' 'A-Star' 'A3' 'A4' 'A6' 'A7' 'A8'
 'Accent' 'Accord' 'Alto' 'Amaze' 'Ameo' 'Aspire' 'Aveo' 'Avventura' 'B'
 'BR-V' 'BRV' 'Baleno' 'Beat' 'Beetle' 'Bolero' 'Bolt' 'Boxster' 'Brio'
 'C-Class' 'CLA' 'CLS-Class' 'CR-V' 'Camry' 'Captiva' 'Captur' 'Cayenne'
 'Cayman' 'Cedia' 'Celerio' 'Ciaz' 'City' 'Civic' 'Classic' 'Clu

127.0.0.1 - - [20/Apr/2023 18:34:54] "POST /submit-form HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 19:06:02] "OPTIONS /submit-form HTTP/1.1" 200 -
C:\Users\bobba\AppData\Local\Temp\ipykernel_16704\2147629206.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\bobba\AppData\Local\Temp\ipykernel_16704\2309633390.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[enc.categories_[0]] = enc.transform(df[i].values.reshape(-1, 1)).toarray()
C:\Users\bobba\AppData\Local\Temp\ipykernel_16704\2309633390.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times,

127.0.0.1 - - [20/Apr/2023 19:06:02] "POST /submit-form HTTP/1.1" 500 -


...

Location <class 'str'> ['Ahmedabad' 'Bangalore' 'Chennai' 'Coimbatore' 'Delhi' 'Hyderabad'
 'Jaipur' 'Kochi' 'Kolkata' 'Mumbai' 'Pune']
Fuel_Type <class 'str'> ['CNG' 'Diesel' 'LPG' 'Petrol']
Transmission <class 'str'> ['Automatic' 'Manual']
Owner_Type <class 'str'> ['First' 'Fourth & Above' 'Second' 'Third']
Cars <class 'str'> ['Ambassador' 'Audi' 'BMW' 'Bentley' 'Chevrolet' 'Datsun' 'Fiat' 'Force'
 'Ford' 'Honda' 'Hyundai' 'ISUZU' 'Isuzu' 'Jaguar' 'Jeep' 'Lamborghini'
 'Land' 'Mahindra' 'Maruti' 'Mercedes-Benz' 'Mini' 'Mitsubishi' 'Nissan'
 'Porsche' 'Renault' 'Skoda' 'Smart' 'Tata' 'Toyota' 'Volkswagen' 'Volvo']
Submodel <class 'str'> ['1' '1000' '3' '5' '6' '7' '800' 'A' 'A-Star' 'A3' 'A4' 'A6' 'A7' 'A8'
 'Accent' 'Accord' 'Alto' 'Amaze' 'Ameo' 'Aspire' 'Aveo' 'Avventura' 'B'
 'BR-V' 'BRV' 'Baleno' 'Beat' 'Beetle' 'Bolero' 'Bolt' 'Boxster' 'Brio'
 'C-Class' 'CLA' 'CLS-Class' 'CR-V' 'Camry' 'Captiva' 'Captur' 'Cayenne'
 'Cayman' 'Cedia' 'Celerio' 'Ciaz' 'City' 'Civic' 'Classic' 'Clu

127.0.0.1 - - [20/Apr/2023 20:32:06] "OPTIONS /submit-form HTTP/1.1" 200 -
C:\Users\bobba\AppData\Local\Temp\ipykernel_16704\2147629206.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\bobba\AppData\Local\Temp\ipykernel_16704\2309633390.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[enc.categories_[0]] = enc.transform(df[i].values.reshape(-1, 1)).toarray()
C:\Users\bobba\AppData\Local\Temp\ipykernel_16704\2309633390.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using

Location <class 'str'> ['Ahmedabad' 'Bangalore' 'Chennai' 'Coimbatore' 'Delhi' 'Hyderabad'
 'Jaipur' 'Kochi' 'Kolkata' 'Mumbai' 'Pune']
Fuel_Type <class 'str'> ['CNG' 'Diesel' 'LPG' 'Petrol']
Transmission <class 'str'> ['Automatic' 'Manual']
Owner_Type <class 'str'> ['First' 'Fourth & Above' 'Second' 'Third']
Cars <class 'str'> ['Ambassador' 'Audi' 'BMW' 'Bentley' 'Chevrolet' 'Datsun' 'Fiat' 'Force'
 'Ford' 'Honda' 'Hyundai' 'ISUZU' 'Isuzu' 'Jaguar' 'Jeep' 'Lamborghini'
 'Land' 'Mahindra' 'Maruti' 'Mercedes-Benz' 'Mini' 'Mitsubishi' 'Nissan'
 'Porsche' 'Renault' 'Skoda' 'Smart' 'Tata' 'Toyota' 'Volkswagen' 'Volvo']
Submodel <class 'str'> ['1' '1000' '3' '5' '6' '7' '800' 'A' 'A-Star' 'A3' 'A4' 'A6' 'A7' 'A8'
 'Accent' 'Accord' 'Alto' 'Amaze' 'Ameo' 'Aspire' 'Aveo' 'Avventura' 'B'
 'BR-V' 'BRV' 'Baleno' 'Beat' 'Beetle' 'Bolero' 'Bolt' 'Boxster' 'Brio'
 'C-Class' 'CLA' 'CLS-Class' 'CR-V' 'Camry' 'Captiva' 'Captur' 'Cayenne'
 'Cayman' 'Cedia' 'Celerio' 'Ciaz' 'City' 'Civic' 'Classic' 'Clu

127.0.0.1 - - [20/Apr/2023 20:32:43] "OPTIONS /submit-form HTTP/1.1" 200 -
C:\Users\bobba\AppData\Local\Temp\ipykernel_16704\2147629206.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\bobba\AppData\Local\Temp\ipykernel_16704\2309633390.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[enc.categories_[0]] = enc.transform(df[i].values.reshape(-1, 1)).toarray()
C:\Users\bobba\AppData\Local\Temp\ipykernel_16704\2309633390.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using

Location <class 'str'> ['Ahmedabad' 'Bangalore' 'Chennai' 'Coimbatore' 'Delhi' 'Hyderabad'
 'Jaipur' 'Kochi' 'Kolkata' 'Mumbai' 'Pune']
Fuel_Type <class 'str'> ['CNG' 'Diesel' 'LPG' 'Petrol']
Transmission <class 'str'> ['Automatic' 'Manual']
Owner_Type <class 'str'> ['First' 'Fourth & Above' 'Second' 'Third']
Cars <class 'str'> ['Ambassador' 'Audi' 'BMW' 'Bentley' 'Chevrolet' 'Datsun' 'Fiat' 'Force'
 'Ford' 'Honda' 'Hyundai' 'ISUZU' 'Isuzu' 'Jaguar' 'Jeep' 'Lamborghini'
 'Land' 'Mahindra' 'Maruti' 'Mercedes-Benz' 'Mini' 'Mitsubishi' 'Nissan'
 'Porsche' 'Renault' 'Skoda' 'Smart' 'Tata' 'Toyota' 'Volkswagen' 'Volvo']
Submodel <class 'str'> ['1' '1000' '3' '5' '6' '7' '800' 'A' 'A-Star' 'A3' 'A4' 'A6' 'A7' 'A8'
 'Accent' 'Accord' 'Alto' 'Amaze' 'Ameo' 'Aspire' 'Aveo' 'Avventura' 'B'
 'BR-V' 'BRV' 'Baleno' 'Beat' 'Beetle' 'Bolero' 'Bolt' 'Boxster' 'Brio'
 'C-Class' 'CLA' 'CLS-Class' 'CR-V' 'Camry' 'Captiva' 'Captur' 'Cayenne'
 'Cayman' 'Cedia' 'Celerio' 'Ciaz' 'City' 'Civic' 'Classic' 'Clu